In [3]:
from IPython.display import IFrame

### Motivation
The purpose of word embeddings is to find a dense representation for each word in a vocabulary. In general, the way the distributed representation of a word is learnt is through a language model task in which each word is encoded as a vector. As an example, the language model task can be to predict a target word given the context around the word. 

Few advantages of a dense representation of words are:
   * Once we have a dense vector representation, these can be used as an input to other models. These typically reduce the input dimention as compared to the case where words are encoded as one-hot-vectors.
   * Using the dense representation, one can compute similartity socre between words. The idea is that "words of a feather flock together". So, if "cat" and "dog" occur in corpus in the same context, vector("cat") and vector("dog") will be similar. 
   * One can use the fact that similar words will have similar vecors to cluster words together. Clustering can be used to acceralate learning tasks as is done in "A scalable hierarchical distributed language model".

A typical neural-network setup which is used to learn a word embedding is shown in the figure below. Here, the "lookup" (red vectors) and the rows of "W" matrix can be considered as word embeddings.  

In [13]:
IFrame("./wordEmbLanguageModel.pdf", width=400, height=300)

### Advantage of non-linearity

Consider the sentences:
   * Paris is a nice place to visit.
   * I stayed at the Hilton.
   * Paris Hilton appeared at the cover of Vogue. 

Note that "Paris" and "Hilton" in the first two sentences come in the context of travel. On the other hand "Paris Hilton" comes in the cotext of celebrity gossip. 

This is a prototypical example of an XOR situation. Such relationships can be learned only by introducing non-linearities in the model.

For the sake of simplicity, we will not be adding non-linearity in the following models. 

### CBOW

One can use a continuous bag of words language model to learn a distributed representation of words. The diagram dipicting the model is shown in the following figure. Note that
   * The words in the context are added and not concatenated. This is ok if our primary task is not to learn the target word itself but just a vector representation of a word. 
   * Words to the right of the target word also come in the context. 

In [21]:
IFrame("./wordEmbLanguageModelCBOW.pdf", width=400, height=300)

### Code implementation

In [149]:
import dynet as dy
from collections import defaultdict
from random import shuffle
import time

In [161]:
# Util function

def get_context(sent, target_idx, n):
    """
    Purpose: Given a sent (list) and a target_idx (int)
             return the context. 
             sent = ["Hello", "There", "I", "Am", "Here"]
             
             target_idx = 0
             return [0, 0, "Hello", "There"]
             
             target_idx = 1
             return [0, "Hello", "There", "I"]
             
             target_idx = 4
             return ["I", "Am", 0, 0]
    """
    
    # Initialize the context
    context = [S] * 2 * n
    
    write_idx = -1
    
    # populate context to the left of the target word
    for idx in range(target_idx - n, target_idx):
        write_idx += 1 
        if idx >= 0:
            context[write_idx] = sent[idx]
            
    # populate context to the right of the target word
    for idx in range(target_idx + 1, target_idx + n + 1):
        write_idx += 1
        if idx < len(sent):
            context[write_idx] = sent[idx]
    return context

In [187]:
class WordEmbCbow(object):
    
    def __init__(self, train_path):
        
        self.N = 2
        self.EMB_SIZE = 32
        
        self.w2i = defaultdict(lambda: len(self.w2i))
        self.S = self.w2i["<s>"]
        self.UNK = self.w2i["<unk>"] 
        self.train = list(self.read_data(train_path))
        self.w2i = defaultdict(lambda: self.UNK, self.w2i)
        self.i2w = {i: w for w, i in self.w2i.iteritems()}
        self.nWords = len(self.w2i)
        
        self.model = dy.Model()
        self.trainer = dy.SimpleSGDTrainer(self.model)
        
        self.W_c_p = self.model.add_lookup_parameters((self.nWords, self.EMB_SIZE)) 
        self.W_w_p = self.model.add_parameters((self.nWords, self.EMB_SIZE))
        
    def read_data(self, filename):
        with open(filename, 'r') as f:
            for line in f:
                yield [self.w2i[word] for word in line.strip().split(" ")]
            
    def calc_sent_loss(self, sent):
        # Create a computation graph
        dy.renew_cg()
        W_w = dy.parameter(self.W_w_p)
    
        sent_loss = []
        for i in range(len(sent)):
            word_idx_in_context_window = get_context(sent, i, self.N)
            context = dy.esum([dy.lookup(self.W_c_p, idx) for idx in word_idx_in_context_window])
            scores = W_w * context
            sent_loss.append(dy.pickneglogsoftmax(scores, i))
        return dy.esum(sent_loss)
    
    def trainCbow(self, max_iter=100):
        
        for ITER in range(max_iter):
            shuffle(self.train)
            train_words = 0
            train_loss = 0.0
            start_time = time.time()
            for sent_id, sent in enumerate(self.train):
                # Each sent is a list of word indexes
                # For each sentence, we calculate the 
                # loss in predicting the target words given the contexts. 
                # We perform a back propagation only once a sentence. 
                sent_loss = self.calc_sent_loss(sent)
                train_loss += sent_loss.value()
                train_words += len(sent) 
                sent_loss.backward()
                self.trainer.update()
                if (sent_id +1) % 1000 == 0:
                    print("Finished processing {} sentences".format(sent_id + 1))
            print("ITER = {}, train_loss/words = {}, time = {}".format(ITER, train_loss/train_words, time.time() - start_time))

In [188]:
cbow = WordEmbCbow("../nn4nlp2017-code-master/data/classes/train.txt")
cbow.trainCbow(1)

Finished processing 1000 sentences
Finished processing 2000 sentences
Finished processing 3000 sentences
Finished processing 4000 sentences
Finished processing 5000 sentences
Finished processing 6000 sentences
Finished processing 7000 sentences
Finished processing 8000 sentences
ITER = 0, train_loss/words = 2.93275829104, time = 89.0067739487


### Skip gram 

There are two ways of looking into Skip gram model:
   * Given the target word, predict the words in the context. This is what is done in the Word2Vec.
   * Treat each word in the context as independant and predict the word given the words in the context. This is what is explained in the book by Goldberg. 
   
The schematic for skip-gram model for predicting the words in the context given a target word is shown below.

In [160]:
IFrame("./wordEmbLanguageModelSG.pdf", width=400, height=300) 

In [183]:
class WordEmbSg(object):
    def __init__(self, train_path):
        
        self.N = 2
        self.EMB_SIZE = 32
        
        self.w2i = defaultdict(lambda: len(self.w2i))
        self.S = self.w2i["<s>"]
        self.UNK = self.w2i["<unk>"] 
        self.train = list(self.read_data(train_path))
        self.w2i = defaultdict(lambda: self.UNK, self.w2i)
        self.i2w = {i: w for w, i in self.w2i.iteritems()}
        self.nWords = len(self.w2i)
        
        self.model = dy.Model()
        self.trainer = dy.SimpleSGDTrainer(self.model)
        
        self.W_c_p = self.model.add_lookup_parameters((self.nWords, self.EMB_SIZE))
        self.W_w_p = self.model.add_parameters((self.nWords, self.EMB_SIZE))
        
    def read_data(self, filename):
        with open(filename, 'r') as f:
            for line in f:
                yield [self.w2i[word] for word in line.strip().split(" ")]
    
    def calc_sent_loss(self, sent):
        # Create a computation graph
        dy.renew_cg()
        
        W_w = dy.parameter(self.W_w_p)
        all_losses = []
        for i in range(len(sent)):
            # Given the target word, calculate the 
            # log_probability of each word in the vocabulary
            score = W_w * dy.lookup(self.W_c_p, sent[i])
            log_prob = dy.log_softmax(score)
            
            # From the computed probability over entire vocabulary
            # select the probability of the words in the context
            word_idx_in_context_window = get_context(sent, i, self.N)
            for context_idx in word_idx_in_context_window:
                all_losses.append(-dy.pick(log_prob, context_idx))
        return dy.esum(all_losses)
    
    def trainSg(self, max_iter=1):
        
        for ITER in range(max_iter):
            shuffle(self.train)
            train_words = 0
            train_loss = 0.0
            start_time = time.time()
            for sent_id, sent in enumerate(self.train):
                # Each sent is a list of word indexes
                # For each sentence, we calculate the 
                # loss in predicting the the contexts given the target word. 
                # We perform a back propagation only once a sentence. 
                sent_loss = self.calc_sent_loss(sent)
                train_loss += sent_loss.value()
                train_words += len(sent) 
                sent_loss.backward()
                self.trainer.update()
                if (sent_id +1) % 1000 == 0:
                    print("Finished processing {} sentences".format(sent_id + 1))
            print("ITER = {}, train_loss/words = {}, time = {}".format(ITER, train_loss/train_words, time.time() - start_time))

In [184]:
sg = WordEmbSg("../nn4nlp2017-code-master/data/classes/train.txt")
sg.trainSg(1)

Finished processing 1000 sentences
Finished processing 2000 sentences
Finished processing 3000 sentences
Finished processing 4000 sentences
Finished processing 5000 sentences
Finished processing 6000 sentences
Finished processing 7000 sentences
Finished processing 8000 sentences
ITER = 0, train_loss/words = 29.2522201881, time = 105.120398045


### Hierarchical softmax

